### 8번 성과정보에서 만든 파생변수 3가지

* 우량후보여부
* 실질여유한도
* 한도대비소비성향

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import platform

# ---------------------------------------------------------
# 1. 폰트 설정
# ---------------------------------------------------------
system_name = platform.system()
if system_name == 'Windows':
    plt.rc('font', family='Malgun Gothic')
elif system_name == 'Darwin':
    plt.rc('font', family='AppleGothic')
else:
    plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False

# ---------------------------------------------------------
# 2. [오류 해결] ID 컬럼 복구 (Index -> Column)
# ---------------------------------------------------------
# merged_df가 정의되어 있다고 가정합니다. (이전 셀 실행 결과)

# 'ID'가 컬럼에 없고, 혹시 인덱스 이름이 'ID'라면 리셋
if 'ID' not in merged_df.columns:
    print("⚠️ 'ID' 컬럼이 보이지 않아 인덱스 초기화를 시도합니다...")
    merged_df = merged_df.reset_index()

# 리셋 후에도 'ID'가 없는 경우 (인덱스 이름이 다른 경우 등)
if 'ID' not in merged_df.columns:
    # 혹시 컬럼명이 'index'로 되어있거나 대소문자가 다른지 확인
    print(f"❌ 여전히 'ID' 컬럼이 없습니다. 현재 컬럼 목록: {list(merged_df.columns)}")
    # 만약 'index'라는 컬럼이 생겼고 그게 ID라면 이름 변경
    if 'index' in merged_df.columns:
        print("ℹ️ 'index' 컬럼을 'ID'로 변경합니다.")
        merged_df.rename(columns={'index': 'ID'}, inplace=True)

# ---------------------------------------------------------
# 3. 데이터 집계 및 시각화 재시도
# ---------------------------------------------------------
if 'ID' in merged_df.columns and 'Segment' in merged_df.columns:
    print("✅ ID 컬럼 확보 완료! 집계 및 시각화를 진행합니다.")

    # (1) ID별 평균 계산
    user_stats_df = merged_df.groupby(['ID', 'Segment'], observed=True).agg({
        '카드이용한도금액': 'mean',
        '잔액_신판최대한도소진율_r3m': 'mean'
    }).reset_index()

    user_stats_df.rename(columns={
        '카드이용한도금액': '평균한도금액',
        '잔액_신판최대한도소진율_r3m': '평균소진율'
    }, inplace=True)

    # (2) 시각화 함수 정의
    def plot_segment_analysis(data, x_col, y_col, title, y_label):
        # 순서 정렬
        if data[x_col].dtype.name != 'category':
            data[x_col] = pd.Categorical(data[x_col], categories=['A', 'B', 'C', 'D', 'E'], ordered=True)
        
        # 통계값 계산
        stats = data.groupby(x_col)[y_col].agg(['count', 'mean', 'min', 'max'])
        
        # 그래프
        fig, ax = plt.subplots(figsize=(12, 9))
        sns.boxplot(x=x_col, y=y_col, data=data, 
                    order=['A', 'B', 'C', 'D', 'E'], 
                    palette='coolwarm', showfliers=False, ax=ax)
        
        ax.set_title(title, fontsize=16, pad=20)
        ax.set_ylabel(y_label, fontsize=12)
        ax.grid(axis='y', linestyle='--', alpha=0.5)
        
        # 표 추가
        cell_text = []
        for row in stats.values:
            if '소진율' in title:
                formatted_row = [f"{int(row[0]):,}", f"{row[1]:.3f}", f"{row[2]:.3f}", f"{row[3]:.3f}"]
            else:
                formatted_row = [f"{x:,.0f}" if i==0 else f"{x:,.0f}" for i, x in enumerate(row)]
            cell_text.append(formatted_row)
            
        the_table = plt.table(cellText=cell_text,
                              rowLabels=stats.index,
                              colLabels=['고객수', '평균', '최소', '최대'],
                              loc='bottom', cellLoc='center',
                              bbox=[0.0, -0.3, 1.0, 0.25])
        
        the_table.auto_set_font_size(False)
        the_table.set_fontsize(11)
        the_table.scale(1, 1.5)

        plt.subplots_adjust(left=0.1, bottom=0.25)
        plt.show()

    # (3) 실행
    # 한도금액
    plot_segment_analysis(user_stats_df, 'Segment', '평균한도금액', 
                          'Segment별 고객 평균 한도금액 분포', '한도금액 (원)')

    # 소진율 (이상치 Cap)
    user_stats_df['시각화용_소진율'] = user_stats_df['평균소진율'].clip(0, 1.2)
    plot_segment_analysis(user_stats_df, 'Segment', '시각화용_소진율', 
                          'Segment별 고객 평균 한도 소진율 (최대소진율 기준)', '소진율 (0.0 ~ 1.2)')

else:
    print("❌ 오류: 'ID' 또는 'Segment' 컬럼을 찾을 수 없습니다.")
    print(f"현재 컬럼 목록: {list(merged_df.columns)}")

In [ ]:
import numpy as np

# ---------------------------------------------------------
# A/B 구분 필살기 파생변수 생성
# ---------------------------------------------------------

# 1. [1차 필터] 우량 고객 후보군 (Premium Candidate)
# 설명: "한도가 5만 원도 안 되면 절대 A, B가 아니다"라는 규칙을 0/1로 박아줍니다.
# 효과: C, D, E 하위 그룹을 1차적으로 걸러냅니다.
threshold_limit = 50000
user_stats_df['우량후보_여부'] = (user_stats_df['평균한도금액'] >= threshold_limit).astype(int)

# 2. [A/B 분리기] 실질 여유 한도 (Real Spare Limit) ★가장 강력함★
# 공식: 평균한도 * (1 - 소진율)
# 설명: "지금 당장 융통할 수 있는 현금 여력"이 얼마인가?
#   - A (부자 & 절약): 한도(大) * 여유(大) = 결과값 [초대형] (예: 13만원)
#   - B (부자 & 낭비): 한도(中) * 여유(小) = 결과값 [중소형] (예: 6만원)
#   - C/D/E: 한도가 작아서 결과값 [소형/영(0)]
user_stats_df['실질여유한도'] = user_stats_df['평균한도금액'] * (1 - user_stats_df['평균소진율'])

# 3. [공격성 지표] 한도 대비 소비성향 (Limit Aggressiveness)
# 공식: 평균소진율 / log(평균한도)
# 설명: 한도가 큰데 소진율까지 높으면(B) 점수가 확 튀게 만듭니다.
#   - A: 분자(소진율)가 낮아서 점수가 낮음 (안정적)
#   - B: 분자(소진율)가 높고 분모(한도)는 A보다 작아서 점수가 높음 (공격적)
# (log를 씌운 이유는 한도 금액의 단위가 너무 커서 소진율이 묻히는 것을 방지하기 위함입니다)
user_stats_df['한도대비_소비성향'] = user_stats_df['평균소진율'] / np.log1p(user_stats_df['평균한도금액'])

# ---------------------------------------------------------
# 생성된 변수 검증 (A와 B가 잘 벌어졌는지 눈으로 확인)
# ---------------------------------------------------------
print("변수 생성 완료! 등급별 평균값 비교:")
check_cols = ['평균한도금액', '평균소진율', '우량후보_여부', '실질여유한도', '한도대비_소비성향']
display(user_stats_df.groupby('Segment')[check_cols].mean().round(4))

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>평균한도금액</th>
      <th>평균소진율</th>
      <th>우량후보_여부</th>
      <th>실질여유한도</th>
      <th>한도대비_소비성향</th>
    </tr>
    <tr>
      <th>Segment</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>A</th>
      <td>197598.8323</td>
      <td>0.2694</td>
      <td>1.0000</td>
      <td>145085.6767</td>
      <td>0.0221</td>
    </tr>
    <tr>
      <th>B</th>
      <td>126400.1250</td>
      <td>0.4108</td>
      <td>1.0000</td>
      <td>77921.2250</td>
      <td>0.0355</td>
    </tr>
    <tr>
      <th>C</th>
      <td>98017.1009</td>
      <td>0.2616</td>
      <td>0.7928</td>
      <td>75935.8600</td>
      <td>inf</td>
    </tr>
    <tr>
      <th>D</th>
      <td>69321.6125</td>
      <td>0.2468</td>
      <td>0.5877</td>
      <td>56116.8822</td>
      <td>inf</td>
    </tr>
    <tr>
      <th>E</th>
      <td>45248.4548</td>
      <td>0.1394</td>
      <td>0.3458</td>
      <td>40933.9013</td>
      <td>inf</td>
    </tr>
  </tbody>
</table>
</div>

In [ ]:

# 1. 장기 우량 고객 (한도 10만 & 입회 10년 이상 가정)
# '입회경과개월수_신용'이 120개월 이상인 사람
df['장기 우량 고객'] = ((df['카드이용한도금액'] >= 100000) & (df['입회경과개월수_신용'] >= 120)).astype(int)

# 2. 클린 신용 지표 (최근 1년 카드론/CA 이용 0건)
# B등급과 A등급을 가르는 핵심
df['클린 신용 지표'] = ((df['이용건수_카드론_R12M'] == 0) & (df['이용건수_CA_R12M'] == 0)).astype(int)

# 3. 할부 의존 지표 (일시불 잔액보다 할부 잔액이 많은 경우)
df['할부 의존 지표'] = (df['잔액_할부_B0M'] > df['잔액_일시불_B0M']).astype(int)

# ---------------------------------------------------------
# [결과 확인] 등급별 이진화 변수 적중률
# ---------------------------------------------------------
check_bin_cols = ['Is_Longterm_Premium', 'Is_Clean_Credit', 'Is_Installment_User']
display(df.groupby('Segment')[check_bin_cols].mean().round(3))

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Is_Longterm_Premium</th>
      <th>Is_Clean_Credit</th>
      <th>Is_Installment_User</th>
    </tr>
    <tr>
      <th>Segment</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>A</th>
      <td>0.769</td>
      <td>0.613</td>
      <td>0.282</td>
    </tr>
    <tr>
      <th>B</th>
      <td>0.208</td>
      <td>0.444</td>
      <td>0.514</td>
    </tr>
    <tr>
      <th>C</th>
      <td>0.197</td>
      <td>0.652</td>
      <td>0.220</td>
    </tr>
    <tr>
      <th>D</th>
      <td>0.081</td>
      <td>0.725</td>
      <td>0.217</td>
    </tr>
    <tr>
      <th>E</th>
      <td>0.027</td>
      <td>0.885</td>
      <td>0.155</td>
    </tr>
  </tbody>
</table>
</div>

In [ ]:
import pandas as pd
import numpy as np

# ---------------------------------------------------------
# 1. [데이터 타입 교정] 모든 분석 대상 컬럼을 숫자로 변환
# ---------------------------------------------------------
# 계산에 사용할 모든 컬럼 리스트
numeric_cols = [
    '잔액_리볼빙일시불이월_B0M', '잔액_리볼빙CA이월_B0M', 
    '캠페인접촉건수_R12M', '변동률_잔액_B1M', '혜택수혜율_R3M'
]

for col in numeric_cols:
    if col in df.columns:
        # 문자열을 숫자로 변환, 숫자가 아닌 값은 NaN 처리 후 0으로 채움
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

# ---------------------------------------------------------
# 2. 파생변수 4종 생성 (이진화)
# ---------------------------------------------------------

# 1. 리볼빙 이월 여부 (B등급 타겟: 자금 압박 신호)
df['리볼빙 이월 여부'] = ((df['잔액_리볼빙일시불이월_B0M'] > 0) | (df['잔액_리볼빙CA이월_B0M'] > 0)).astype(int)

# 2. 잔액 급증 신호 (위험 감지: 급격한 부채 증가)
df['잔액 급증 신호'] = (df['변동률_잔액_B1M'] > 0.5).astype(int)

# 3. 혜택 수혜 우수자 (평균보다 혜택을 많이 챙기는 그룹)
benefit_mean = df['혜택수혜율_R3M'].mean()
df['혜택 고수혜 여부'] = (df['혜택수혜율_R3M'] > benefit_mean).astype(int)

# ---------------------------------------------------------
# 3. 결과 확인 (발표용 표 출력)
# ---------------------------------------------------------
print("📊 [Segment별 파생변수 비중 요약]")
check_cols = ['Is_Revolving_User', 'Is_Balance_Surge', 'Is_Benefit_High']

# 등급별 평균값 계산 (해당 특성을 가진 고객의 비율)
result_table = df.groupby('Segment', observed=True)[check_cols].mean()

# % 포맷으로 깔끔하게 출력
display(result_table.round(3).style.format("{:.1%}"))

<style type="text/css">
</style>
<table id="T_8284f">
  <thead>
    <tr>
      <th class="blank level0" >&nbsp;</th>
      <th id="T_8284f_level0_col0" class="col_heading level0 col0" >Is_Revolving_User</th>
      <th id="T_8284f_level0_col1" class="col_heading level0 col1" >Is_Balance_Surge</th>
      <th id="T_8284f_level0_col2" class="col_heading level0 col2" >Is_Benefit_High</th>
    </tr>
    <tr>
      <th class="index_name level0" >Segment</th>
      <th class="blank col0" >&nbsp;</th>
      <th class="blank col1" >&nbsp;</th>
      <th class="blank col2" >&nbsp;</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th id="T_8284f_level0_row0" class="row_heading level0 row0" >A</th>
      <td id="T_8284f_row0_col0" class="data row0 col0" >4.3%</td>
      <td id="T_8284f_row0_col1" class="data row0 col1" >0.2%</td>
      <td id="T_8284f_row0_col2" class="data row0 col2" >0.2%</td>
    </tr>
    <tr>
      <th id="T_8284f_level0_row1" class="row_heading level0 row1" >B</th>
      <td id="T_8284f_row1_col0" class="data row1 col0" >8.3%</td>
      <td id="T_8284f_row1_col1" class="data row1 col1" >0.0%</td>
      <td id="T_8284f_row1_col2" class="data row1 col2" >0.0%</td>
    </tr>
    <tr>
      <th id="T_8284f_level0_row2" class="row_heading level0 row2" >C</th>
      <td id="T_8284f_row2_col0" class="data row2 col0" >4.8%</td>
      <td id="T_8284f_row2_col1" class="data row2 col1" >0.7%</td>
      <td id="T_8284f_row2_col2" class="data row2 col2" >0.8%</td>
    </tr>
    <tr>
      <th id="T_8284f_level0_row3" class="row_heading level0 row3" >D</th>
      <td id="T_8284f_row3_col0" class="data row3 col0" >5.4%</td>
      <td id="T_8284f_row3_col1" class="data row3 col1" >0.7%</td>
      <td id="T_8284f_row3_col2" class="data row3 col2" >1.5%</td>
    </tr>
    <tr>
      <th id="T_8284f_level0_row4" class="row_heading level0 row4" >E</th>
      <td id="T_8284f_row4_col0" class="data row4 col0" >2.2%</td>
      <td id="T_8284f_row4_col1" class="data row4 col1" >1.1%</td>
      <td id="T_8284f_row4_col2" class="data row4 col2" >9.7%</td>
    </tr>
  </tbody>
</table>